<div style='text-align:center'>
    <h1 style='font-size: 60px; margin-bottom: 1rem'>GWANW 2024 Demo</h1>
    <p style='font-size: 25px;'>A (live 😱) demo of some of the capabilities of LEGWORK</p>
</div>

This notebook is just a list of headings, for the full code that goes with them check out the separate notebooks in the folder! Below are some imports that I'll need later

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from ipywidgets import interact, FloatSlider

import legwork as lw

# Demo 1: Interactive SNR calculations

#### An intro to `Source`

#### A function to print the SNR and merger time of a single source

#### Interactive sliders

# Demo 2: Evolving source populations

#### Initialise a random set of sources

#### Some plotting routines
- How does the mass distribution change for detectable sources?

#### Calculate merger times
- Compare (`f_orb`, `ecc`) with the merger times

#### Evolve sources
- Compare how `f_orb` and `ecc` have changed

### Evolve sources different times

# Demo 3 - Compare sensitivity curves

#### Plot a sensitivity curve

#### LISA variations (observation time, arm length)

#### Different detectors

#### Calculating SNRs in each

- Avoid interpolating the SC
- Compare LISA and TianQin
- Custom instrument